1. Import the necessary libraries

In [20]:
# Import necessary libraries
import pandas as pd
import numpy as np

2. Load the dataset

In [23]:
# Load the dataset
path = 'C:/Users/Jacques/OneDrive/Documents/Data Analytics course/Data Immersion/Section 6/lending_club_loans.csv'
# Load the CSV into a DataFrame
# Using low_memory=False to avoid DtypeWarning on mixed-type columns
df = pd.read_csv(path, low_memory=False)

In [24]:
# Preview the first few rows
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


3. Clean the dataset

In [28]:
# Drop irrelevant columns (e.g., identifiers)
df.drop(columns=['id', 'member_id'], inplace=True)

In [30]:
# Handle missing values
# Drop columns with more than 90% missing values
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.9].index
df.drop(columns=cols_to_drop, inplace=True)

In [32]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

In [36]:
# Convert data types
# Convert interest rate from string to float safely
df['int_rate'] = (
    df['int_rate']
    .astype(str)                     # convert to string first
    .str.rstrip('%')                # remove the percent sign
    .replace('nan', np.nan)         # convert 'nan' string back to actual NaN
    .astype(float) / 100            # convert to float and divide to make it a decimal
)

In [38]:
# Preview results
df['int_rate'].head()

0    0.1399
1    0.1199
2    0.1078
3    0.1485
4    0.2245
Name: int_rate, dtype: float64

In [42]:
# Convert key categorical columns to category data type
categorical_cols = ['term', 'grade', 'sub_grade', 'home_ownership', 
                    'verification_status', 'purpose', 'loan_status']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

In [44]:
# Rename columns for clarity
df.rename(columns={'annual_inc': 'annual_income'}, inplace=True)

In [48]:
# Convert date columns to datetime format
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

In [50]:
# Consistency checks
# Preview loan status values
print("Loan Status Counts:")
print(df['loan_status'].value_counts())

# Compare related loan amount fields
print("\nSummary of loan amount fields:")
print(df[['loan_amnt', 'funded_amnt', 'funded_amnt_inv']].describe())

Loan Status Counts:
loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

Summary of loan amount fields:
          loan_amnt   funded_amnt  funded_amnt_inv
count  2.260668e+06  2.260668e+06     2.260668e+06
mean   1.504693e+04  1.504166e+04     1.502344e+04
std    9.190245e+03  9.188413e+03     9.192332e+03
min    5.000000e+02  5.000000e+02     0.000000e+00
25%    8.000000e+03  8.000000e+03     8.000000e+03
50%    1.290000e+04  1.287500e+04    

In [52]:
# Final overview
print("\nFinal shape of cleaned dataset:", df.shape)
df.info()


Final shape of cleaned dataset: (2260669, 112)
<class 'pandas.core.frame.DataFrame'>
Index: 2260669 entries, 0 to 2260698
Columns: 112 entries, loan_amnt to debt_settlement_flag
dtypes: category(7), datetime64[ns](1), float64(88), object(16)
memory usage: 1.8+ GB


4. Understanding the dataset

In [57]:
# Descriptive statistics

# Continous variables
df[['loan_amnt', 'int_rate', 'installment', 'annual_income', 'dti']].describe()

# Categorical variables
df['loan_status'].value_counts(normalize=True)

loan_status
Fully Paid                                             0.476298
Current                                                0.388521
Charged Off                                            0.118796
Late (31-120 days)                                     0.009496
In Grace Period                                        0.003732
Late (16-30 days)                                      0.001924
Does not meet the credit policy. Status:Fully Paid     0.000879
Does not meet the credit policy. Status:Charged Off    0.000337
Default                                                0.000018
Name: proportion, dtype: float64